# 〜AIを用いて toto予想 をしてみる〜

### ・totoとは

totoとは、指定されたJリーグの試合結果を予想するスポーツくじ。

1口100円からの購入が可能で、的中すると最高当せん金額は最高5億円！

https://www.toto-dream.com/toto/index.html?cid=to_toto_top_bn_bt2

### ・なぜtoto予想？

私は学生時代に10年程サッカーの経験があります。  
その知識や経験を用いて特徴量を作り、試合結果を予測できるのでは？と思いました。  
そして、「サッカーの試合結果を予測してみた」よりも  
「toto予想してみた」の方が面白そうなことをしていると思われそうなので、  
テーマをtoto予想に決定しました。  
~~というか、単純に宝くじの結果を予測して儲けたいなと思いました。~~

### ・最終的な目標

今回目指すゴールは、  
<br>
**「AIを用いてサッカーの試合結果を予想する！！  
その結果を予想とtotoの結果を比較してみる！！  
~~totoを実際に購入してお金を増やす！！~~」**  
<br>
です！  
純粋な気持ちで試合結果を予想します！  
下心はありません！

### ・モデルの精度

どの程度のモデル精度を目指すのかを決めていきます。  
<br>
まずはtotoについて調べてみます。  
https://www.toto-dream.com/toto/about/index.html  
公式サイトによると、totoには大きく3種類のくじがあります。

|名前|金額|予想する内容|一等|条件|二等|条件|三等|条件|
|-|-|-|-|-|-|-|-|-|
|toto|¥100|13試合の勝敗|¥28,806,105|全的中|¥1,010,209|1試合外れ|¥57,240|2試合外れ|
|mini toto|¥100|5試合の勝敗|￥15,068|全的中|-|-|-|-|
|toto GOAL3|¥100|3試合（6チーム)の得点|¥165,798|全的中|¥4,288|1チーム外れ|-|-|

*※当せん金はくじの売上げ、キャリーオーバーの有無により変動するため、  
過去の当せん金額の平均値を当せん金とする。  
下記参考*  
http://san3san.hatenablog.jp/entry/toto_average  

また、totoには「ダブル」「トリプル」という買い方があります。  
下記参考  
https://toto-yoso.com/double_triple/  
<br>
<br>
上記を踏まえて、~~どのくじをどのように購入するか、  
利益を出すために必要な~~モデルの精度を考えていきます。  
くじの種類、モデルの精度、買い方のあらゆる組み合わせについて  
期待値、必要な金額を算出し、現実的な目標設定を行います。  
手計算は面倒なので上記を計算するコードを書いていきます。

In [89]:
import numpy as np
from numpy import *
import random
import pandas as pd
from fractions import Fraction
from operator import mul
from functools import reduce

In [90]:
"""
totoの計算色々
"""
class TotoCalc:
    
    def __init__(self, n_total_match, n_branch, awards):
        
        self.n_total_match = n_total_match # 総試合数(予想する数)
        self.n_branch = n_branch # 予想の分岐数（勝、負、引の3通りを予想する場合は3）
        self.award1, self.award2, self.award3 = awards # n等の当せん金額
        self.price = 100 # くじの金額
    
    """
    totoの計算色々
    
    precision : モデルの精度（一試合あたりの当たる確率）（０〜１）
    n_double : ダブルで買う試合数
    n_triple : トリプルで買う試合数
    """
    def calc_toto(self, precision, n_double=0, n_triple=0):
        
        # 購入金額
        purchase_price = self.calc_purchase_price(n_double, n_triple)
        # 1等が当たる確率
        p_hit_1, f_p_hit_1 = self.calc_hit(precision, n_double, n_triple, self.n_total_match)
        # 2等が当たる確率
        p_hit_2, f_p_hit_2 = self.calc_hit(precision, n_double, n_triple, self.n_total_match-1)
        # 3等が当たる確率
        p_hit_3, f_p_hit_3 = self.calc_hit(precision, n_double, n_triple, self.n_total_match-2)
        
        # 期待値と当選確率
        if self.award2 == 0 and self.award3 == 0:
            expected_value = self.award1*p_hit_1
            p_hit = p_hit_1
        elif self.award3 == 0:
            expected_value = self.award1*p_hit_1 + self.award2*p_hit_2
            p_hit = p_hit_1+p_hit_2
        else:
            expected_value = self.award1*p_hit_1 + self.award2*p_hit_2 + self.award3*p_hit_3
            p_hit = p_hit_1+p_hit_2+p_hit_3
        
        f_p_hit = self._fraction(p_hit)
        
        # 期待値, 当選確率, 購入金額, 一等の確率, 二等の確率, 三等の確率
        return expected_value, f_p_hit, purchase_price, f_p_hit_1, f_p_hit_2, f_p_hit_3
        
        
    """
    必要な購入金額を算出する
    """
    def calc_purchase_price(self, n_double, n_triple):
        return self.price * (2**n_double) * (3**n_triple)
    
    """
    当選確率を求める
    n_hit : 予想を的中させる試合数
    """
    def calc_hit(self, precision, n_double, n_triple, n_hit):
        
        p_hit = 1 # 当選確率
        n_double_tmp = n_double
        n_triple_tmp = n_triple
        n_hit_tmp = n_hit
        
        for i in range(self.n_total_match):
            
            if n_double_tmp == 0 and n_triple_tmp == 0:
                p_i_hit = precision # p_i_hit : i 試合目の的中確率
            elif 0 < n_double_tmp:
                p_i_hit = precision + (1-precision) / (self.n_branch-1)
                n_double_tmp -= 1
            else:
                p_i_hit = precision + ((1-precision) / (self.n_branch-1))*2
                n_triple_tmp -= 1
            
            if n_hit_tmp == 0:
                n = 1-p_i_hit
            else:
                n = p_i_hit
                n_hit_tmp -= 1
            
            p_hit = p_hit*n
        # 当選確率
        p_hit = p_hit*self._cmb(self.n_total_match, n_hit)
        # 当選確率を分数に変換（1/n)
        f_p_hit = self._fraction(p_hit)
        
        return p_hit, f_p_hit
    
    """
    少数を分数に変換する
    """
    def _fraction(self, f):
        if f <= 0:
            return 0
        a = Fraction(f).limit_denominator(100000000)
        # n = round(a.denominator/a.numerator)
        n = a.denominator/a.numerator
        return Fraction(1/n).limit_denominator(100000000)
    
    """
    nCrを計算する
    """
    def _cmb(self, n, r):
        r = min(n-r,r)
        if r == 0: return 1
        over = reduce(mul, range(n, n - r, -1))
        under = reduce(mul, range(1,r + 1))
        return over // under

In [91]:
# 結果確認用のクラス
class Show_result():
    def __init__(self):
        self.cols = ["くじ種", "期待値", "精度", "当選確率", "購入金額", "ダブル", "トリプル", "一等の確率", "二等の確率", "三等の確率"]
        self.result_df = pd.DataFrame(index=[], columns=self.cols)

    def add_result(self, kind, exp, act, p, place, w, t, p1, p2, p3):
        record = pd.Series([kind, exp, act, p, place, w, t, p1, p2, p3], index=self.result_df.columns)
        self.result_df = self.result_df.append(record, ignore_index=True)
    
    def show_result(self):
        pd.options.display.float_format = '{:.2f}'.format
        return self.result_df.sort_values('期待値', ascending=False)

In [99]:
"""
ダブル、トリプル、精度について
クロスバリデーションにて各計算結果を算出する
"""
def totoSimulate(name, toto, start_precision, step, show_result):
    
    max_double = 8
    max_triple = 5
    max_precision = 0.9
    if max_precision < start_precision:
        print("err")
        return "err"
    
    for n_d in range(max_double+1):
        for n_t in range(max_triple+1):
            
            precision = start_precision
            
            while precision <= max_precision:
                
                do_flg = False
                # ダブル、トリプルはくじごとに使用回数が決められている
                if name == "toto":
                    if (n_t == 0 and n_d <= 8)or(n_t == 1 and n_d <= 7)or(n_t == 2 and n_d <= 5)or(n_t == 3 and n_d <= 4)or(n_t == 4 and n_d <= 2)or(n_t == 5 and n_d <= 1):
                        do_flg = True
                elif name == "mini_toto":
                    if n_d+n_t <= 5:
                        do_flg = True
                elif name == "toto_goal3":
                    if n_d+n_t <= 6:
                        do_flg = True
                
                if do_flg:
                    # 期待値, 当選確率, 購入金額, 一等の確率, 二等の確率, 三等の確率
                    exp, p, place, p1, p2, p3 = toto.calc_toto(precision, n_double = n_d, n_triple = n_t)
                    # くじ種, 期待値, 精度, 当選確率, 購入金額, ダブル, トリプル, 一等の確率, 二等の確率, 三等の確率
                    show_result.add_result(name, exp, precision, p, place, n_d, n_t, p1, p2, p3)
                    # print(" 実行済：",name ,n_d ,n_t ,precision)
                
                precision = round(precision+step, 2)


In [93]:
n_total_match = 13
n_branch = 3
awards = 28806105, 1010209, 57240
s_precision_toto = 1/n_branch
toto = TotoCalc(n_total_match, n_branch, awards)

n_total_match = 6
n_branch = 4
awards = 165798, 4288, 0
s_precision_totogoal3 = 1/n_branch
toto_goal3 = TotoCalc(n_total_match, n_branch, awards)

n_total_match = 5
n_branch = 3
awards = 15068, 0, 0
s_precision_minitoto = 1/n_branch
mini_toto = TotoCalc(n_total_match, n_branch, awards)

In [94]:
show_result = Show_result()
step = 0.01

totoSimulate("toto", toto, s_precision_toto, step, show_result)
totoSimulate("mini_toto", mini_toto, s_precision_totogoal3, step, show_result)
totoSimulate("toto_goal3", toto_goal3, s_precision_minitoto, step, show_result)

**計算出来ました。結果を見てみます**

In [95]:
show_result.show_result()

,くじ種,期待値,精度,当選確率,購入金額,ダブル,トリプル,一等の確率,二等の確率,三等の確率
695,toto,13777052.83,0.90,112582812/72715477,48600,1,5,18357349/40400691,59657199/90895178,19885733/45447589
1449,toto,13124556.30,0.90,145301010/98513351,43200,4,3,43159125/99706427,21209391/33921721,14139594/33921721
985,toto,13088200.19,0.90,43631699/29664244,32400,2,4,34287246/79430597,42361801/67940509,25853127/62195465
347,toto,13051944.79,0.90,36669429/25000000,24300,0,5,35029837/81376319,62178597/100000000,20726199/50000000
694,toto,12747511.55,0.89,15376449/9685226,48600,1,5,30850009/73806301,31926427/47538167,12526271/25151274
...,...,...,...,...,...,...,...,...,...,...
1918,mini_toto,30.91,0.29,199621/97323168,100,0,0,199621/97323168,2172229/86513775,2353769/19144902
1917,mini_toto,25.93,0.28,16807/9765625,100,0,0,16807/9765625,43218/1953125,222264/1953125
1916,mini_toto,21.62,0.27,63872/44513495,100,0,0,63872/44513495,1485182/76565259,9292741/88594512
1915,mini_toto,17.90,0.26,112277/94498314,100,0,0,112277/94498314,1056757/62500000,3007693/31250000


**このようになりました。  
ここから現実的に達成できそうなものを選んでいきます。  
<br>
同じように機械学習でtoto予想をしている方達のブログ等を色々みたところ、  
どれも精度は5割以下でした。  
目標を高く持って、精度5割を目指します。  
<br>
~~あと、お財布と相談の結果、軍資金を5000円とします。~~**

In [96]:
df = show_result.show_result()
df_1 = df[df['精度'] < 0.50]
df_1 = df_1[df_1['購入金額'] < 5000]

In [97]:
df_1

,くじ種,期待値,精度,当選確率,購入金額,ダブル,トリプル,一等の確率,二等の確率,三等の確率
1292,toto,48435.10,0.49,10074473/99373598,4800,4,1,63758/62278903,1254723/90580910,5571726/64409989
828,toto,42760.60,0.49,1156049/12916397,3600,2,2,32978/36487747,1185279/96922783,2067193/27068299
1291,toto,40790.74,0.48,7930675/88022628,4800,4,1,83703/99056789,871979/73272969,7586687/98079123
190,toto,37750.90,0.49,7007973/88689872,2700,0,3,60148/75380739,1027401/95161600,5678189/84218257
1466,toto,36084.15,0.49,2904153/38451377,3200,5,0,19131/25083473,203023/19673326,2506091/38886964
827,toto,35755.22,0.48,1287751/16305656,3600,2,2,16676/22514241,957167/91758791,1607249/23704444
1290,toto,34254.31,0.47,942463/11809122,4800,4,1,11923/17157473,214830/21088289,916663/13299241
1060,toto,31856.65,0.49,5894884/88406487,2400,3,1,48733/72375172,322448/35392293,3542298/62259949
189,toto,31341.31,0.48,4868245/70323586,2700,0,3,61438/94629097,768891/84090502,4971417/83646737
1465,toto,30185.15,0.48,6495903/97429840,3200,5,0,629/1005917,803787/91273987,5691904/99437513


**上記の表は期待値でソートしてあります。  
理論値とはいえ、精度5割~~・軍資金5000円~~でも十分結果を期待できることが分かりました。**

## ・まとめ

**今回の目標をまとめると、  
<br>
*①サッカーの試合結果を5割以上の精度で予想できるモデルを作成する。  
~~②予想結果を元にtoto（ダブル４、トリプル１、計4800円）を購入する。  
③当選する！~~*  
<br>
になりました！！**  
<br>
この後は学習に使えそうなデータを片っ端からスクレイピングしていきます！  
モデルの詳細はその後に考えることとします。  

終わり

#### ・おまけ

In [98]:
df_1.sort_values('当選確率', ascending=False)

,くじ種,期待値,精度,当選確率,購入金額,ダブル,トリプル,一等の確率,二等の確率,三等の確率
4592,toto_goal3,14861.15,0.49,19893485/35581863,4800,4,1,2393616/31017341,3493576/7249281,18877216/76498311
4591,toto_goal3,14001.46,0.48,52921606/97602473,4800,4,1,6738409/93206551,38019001/80905196,20230421/82127568
4590,toto_goal3,13182.80,0.47,4133188/7864927,4800,4,1,4937742/72968083,32532733/71055194,19559113/79635280
4128,toto_goal3,13875.18,0.49,5509306/10554275,3600,2,2,6871062/95364701,10129392/22512401,86995846/74305823
4589,toto_goal3,12403.62,0.46,26950005/52944808,4800,4,1,4750355/75064481,43103959/96702656,11943936/48828125
4127,toto_goal3,13015.93,0.48,38225561/75836738,3600,2,2,1046228/15567303,24263108/55541821,95429215/80659016
4588,toto_goal3,11662.43,0.45,6385729/12960000,4800,4,1,4212549/71245631,39083371/90137158,15768841/64800000
3490,toto_goal3,12954.63,0.49,42379843/86957001,2700,0,3,156848/2331613,29684035/70660234,100217981/91681916
4126,toto_goal3,12198.89,0.47,46173825/94949522,3600,2,2,5264676/84074363,17347525/40944936,114309565/95703369
4587,toto_goal3,10957.76,0.44,35555329/74594388,4800,4,1,2627428/47606165,34701629/82337113,15457703/63922319


当選確率でソートすると、toto Goal3 が一番当たりやすいという結果でした。  
その確率はなんと1/2を超えています。  
当選金額は少なくてもいいからとりあえず当選して喜びたい場合には、  
toto Goal3 が良さそうです。  
このくじは試合結果ではなくチームの得点数を予想するくじなので、  
今回の目的変数は得点数にした方が後々良さそうです。  
（得点数がわかれば試合結果もわかるので）  
ということを頭の片隅に置いて今後の作業を続けていきたいと思います。